In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, contains, when, sum, avg, round,max, row_number, desc
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType

In [4]:
spark = SparkSession.builder.appName("ExtractNestedData").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/06/25 23:23:20 WARN Utils: Your hostname, Benjamins-MacBook-Air.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.35 instead (on interface en0)
25/06/25 23:23:20 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/25 23:23:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [60]:
df = spark.read.csv("/Users/benjaminbrooke/Desktop/Data/dicubed_sdtm_csv_2019-05-30/tr.csv",header = True)
df = df.drop("_c0")

In [61]:
df = df.withColumn("id",row_number().over(Window.partitionBy("STUDYID").orderBy("TRDTC")))

In [62]:
df.show()

+--------------------+------+----------+-----+-------+--------+----------------+-------+--------+--------+--------+--------+--------+--------+-----+----------+---+
|             STUDYID|DOMAIN|   USUBJID|TRSEQ|TRLNKID|TRTESTCD|          TRTEST|TRORRES|TRORRESU|TRSTRESC|TRSTRESN|TRSTRESU|TRMETHOD|VISITNUM|VISIT|     TRDTC| id|
+--------------------+------+----------+-----+-------+--------+----------------+-------+--------+--------+--------+--------+--------+--------+-----+----------+---+
|Breast-MRI-NACT-P...|    TR|UCSF-BR-09|   41|    T01|  VOLUME|          Volume|  29.90|      mL|   29.90|    29.9|      mL|     MRI|       1|  MR1|1986-11-26|  1|
|Breast-MRI-NACT-P...|    TR|UCSF-BR-09|   42|    T01|   LDIAM|Longest Diameter|  64.00|      mm|   64.00|      64|      mm|     MRI|       1|  MR1|1986-11-26|  2|
|Breast-MRI-NACT-P...|    TR|UCSF-BR-09|   43|    T01|  VOLUME|          Volume|  20.00|      mL|   20.00|      20|      mL|     MRI|       3|  MR3|1987-01-27|  3|
|Breast-MRI-NACT

In [63]:
df = df.drop("STUDYID", "DOMAIN")

In [9]:
df.show()

+----------+-----+-------+--------+----------------+-------+--------+--------+--------+--------+--------+--------+-----+----------+---+
|   USUBJID|TRSEQ|TRLNKID|TRTESTCD|          TRTEST|TRORRES|TRORRESU|TRSTRESC|TRSTRESN|TRSTRESU|TRMETHOD|VISITNUM|VISIT|     TRDTC| id|
+----------+-----+-------+--------+----------------+-------+--------+--------+--------+--------+--------+--------+-----+----------+---+
|UCSF-BR-09|   41|    T01|  VOLUME|          Volume|  29.90|      mL|   29.90|    29.9|      mL|     MRI|       1|  MR1|1986-11-26|  1|
|UCSF-BR-09|   42|    T01|   LDIAM|Longest Diameter|  64.00|      mm|   64.00|      64|      mm|     MRI|       1|  MR1|1986-11-26|  2|
|UCSF-BR-09|   43|    T01|  VOLUME|          Volume|  20.00|      mL|   20.00|      20|      mL|     MRI|       3|  MR3|1987-01-27|  3|
|UCSF-BR-09|   44|    T01|   LDIAM|Longest Diameter|  64.00|      mm|   64.00|      64|      mm|     MRI|       3|  MR3|1987-01-27|  4|
|UCSF-BR-67|  369|    T01|   LDIAM|Longest Diame

# First way of finding the percentage change

In [64]:
df_first = df.filter(col("TRTEST")== "Volume").drop("id")

In [65]:
df_first.show()

+----------+-----+-------+--------+------+-------+--------+--------+--------+--------+--------+--------+-----+----------+
|   USUBJID|TRSEQ|TRLNKID|TRTESTCD|TRTEST|TRORRES|TRORRESU|TRSTRESC|TRSTRESN|TRSTRESU|TRMETHOD|VISITNUM|VISIT|     TRDTC|
+----------+-----+-------+--------+------+-------+--------+--------+--------+--------+--------+--------+-----+----------+
|UCSF-BR-01|    2|    T01|  VOLUME|Volume|   9.00|      mL|    9.00|       9|      mL|     MRI|       1|  MR1|1990-03-29|
|UCSF-BR-01|    4|    T01|  VOLUME|Volume|   4.40|      mL|    4.40|     4.4|      mL|     MRI|       2|  MR2|1990-04-17|
|UCSF-BR-01|    5|    T01|  VOLUME|Volume|   0.30|      mL|    0.30|     0.3|      mL|     MRI|       3|  MR3|1990-07-03|
|UCSF-BR-02|    8|    T01|  VOLUME|Volume|  44.30|      mL|   44.30|    44.3|      mL|     MRI|       1|  MR1|1989-11-08|
|UCSF-BR-02|   10|    T01|  VOLUME|Volume|  22.20|      mL|   22.20|    22.2|      mL|     MRI|       3|  MR3|1990-02-07|
|UCSF-BR-03|   11|    T0

In [66]:
df_first = df_first.withColumn("TRSTRESN",col("TRSTRESN").cast("float"))

In [67]:
from pyspark.sql.functions import lag, col, first
windowSpec = Window.partitionBy("USUBJID").orderBy(col("VISITNUM"))
windowSpec_2 = Window.partitionBy("USUBJID","VISITNUM")

In [68]:
df_first = df_first.withColumn("LAG_TRORRES",lag("TRORRES", -1).over(windowSpec))

In [71]:
df_first = df_first.withColumn("Percentage_Change", 
                   first(round(((col("TRSTRESN") - lag(col("TRSTRESN"), 1).over(windowSpec)) / 
                               lag("TRSTRESN", 1).over(windowSpec))*100,2)).over(windowSpec_2)).drop(

In [72]:
df_first.show()

+----------+-----+-------+--------+------+-------+--------+--------+--------+--------+--------+--------+-----+----------+-----------+-----------------+
|   USUBJID|TRSEQ|TRLNKID|TRTESTCD|TRTEST|TRORRES|TRORRESU|TRSTRESC|TRSTRESN|TRSTRESU|TRMETHOD|VISITNUM|VISIT|     TRDTC|LAG_TRORRES|Percentage_Change|
+----------+-----+-------+--------+------+-------+--------+--------+--------+--------+--------+--------+-----+----------+-----------+-----------------+
|UCSF-BR-01|    2|    T01|  VOLUME|Volume|   9.00|      mL|    9.00|     9.0|      mL|     MRI|       1|  MR1|1990-03-29|       4.40|             NULL|
|UCSF-BR-01|    4|    T01|  VOLUME|Volume|   4.40|      mL|    4.40|     4.4|      mL|     MRI|       2|  MR2|1990-04-17|       0.30|           -51.11|
|UCSF-BR-01|    5|    T01|  VOLUME|Volume|   0.30|      mL|    0.30|     0.3|      mL|     MRI|       3|  MR3|1990-07-03|       NULL|           -93.18|
|UCSF-BR-02|    8|    T01|  VOLUME|Volume|  44.30|      mL|   44.30|    44.3|      mL|  